In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from pathlib import Path
import sys, os
sys.path.append(os.path.join(os.path.abspath(""), '..', 'pipeline'))
import sic_matcher as sm

In [2]:
# establishing the file path in such a way to work universally
here = Path().resolve()

In [3]:
# change either code to your desired code
SIC_CODE = "2015" # Poultry Slaughtering and Processing
NAICS_CODE = "311615" # Poultry Processing

In [4]:
FSIS_FILEPATH = "data/fsis-processors-with-location.csv"
FSIS_DATA = here.parent / FSIS_FILEPATH

IG_FILEPATH = "data/2022_Business_Academic_QCQ.txt"
INFOGROUP_2022 = [here.parent / IG_FILEPATH]

## Method Using Dask DataFrames

In [5]:
# def dask_sic_matches_df(mst_df, sic_code):
    
#     """
#         function will filter the master dataframe input that contains everything read in from csv file.
#         This will filter the master dataframe to contain only those rows that contain the SIC Code that
#         is input into the function
        
#         Args:
#             mst_df (dataframe): master dataframe resulted from reading in the csv file
#             sic_code  (string): SIC Code that the user 
            
#         Returns:
#             a dataframe that contains only the rows where the input SIC Code is listed for that row (busienss)
#     """
    
#     # reassure SIC Code is a string    
#     sic_code = str(sic_code)
    
#     # this filters checks SIC Code 1 through 4 & Primary SIC Code if they contain desired SIC Code we are searching for
#     filtered_df = mst_df[ mst_df['SIC CODE'].str.contains(sic_code, na=False) |
#                           mst_df['SIC CODE 1'].str.contains(sic_code, na=False) |
#                           mst_df['SIC CODE 2'].str.contains(sic_code, na=False) |
#                           mst_df['SIC CODE 3'].str.contains(sic_code, na=False) |
#                           mst_df['SIC CODE 4'].str.contains(sic_code, na=False) |
#                           mst_df['PRIMARY SIC CODE'].str.contains(sic_code, na=False)
#                          ]
    
#     result = filtered_df.compute()
            
#     return result

In [5]:
# Read in csv file using Dask dataframe (rather than pandas)
df = dd.read_csv(INFOGROUP_2022[0], dtype=str, encoding='unicode_escape')
df.columns = df.columns.str.upper() # force all columns to be uppercase
df.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,...,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,MIRAGE DRAPERIES,4731 E SUNNY DUNES RD,PALM SPRINGS,CA,92264,1536,065,760,2,A,...,6,044804,1,033.812119,-116.494500,0,40140,2,348,06065
1,NaN,11700 OLIO RD,FISHERS,IN,46037,7618,057,317,1,A,...,7,110808,1,039.959999,-085.921268,P,26900,2,294,18057
2,STONE RESIN SURFACING LLC,1 MAYFLOWER PL,MILFORD,CT,06460,4520,009,203,2,D,...,7,150200,2,041.207739,-073.070136,P,35300,2,408,09009
3,NaN,2121 7TH ST,PARKERSBURG,WV,26101,3803,107,740,1,NaN,...,7,000702,1,039.269405,-081.520599,P,37620,2,425,54107
4,IRONWOOD STATE PRISON,19005 WILEYS WELL RD,BLYTHE,CA,92225,2287,065,760,2,A,...,7,046900,1,033.570065,-114.898031,0,40140,2,348,06065


In [6]:
selected_code = sm.dask_sic_matches_df(df, SIC_CODE)

In [31]:
FILEPATH = "data/code_" + SIC_CODE + ".csv"
selected_code.to_csv(here.parent / FILEPATH) # save as CSV file

## Panda Dataframe Method

In [7]:
# read in entire Infrogroup 2022 file
df = pd.read_csv(INFOGROUP_2022[0], dtype=str)

In [4]:
# covert all the elements of these columns to strings
df['SIC CODE'] = df['SIC CODE'].astype(str)
df['SIC CODE 1'] = df['SIC CODE 1'].astype(str)
df['SIC CODE 2'] = df['SIC CODE 2'].astype(str)
df['SIC CODE 3'] = df['SIC CODE 3'].astype(str)
df['SIC CODE 4'] = df['SIC CODE 4'].astype(str)
df['PRIMARY SIC CODE'] = df['PRIMARY SIC CODE'].astype(str)
            

In [5]:
# functin to build a dataframe of rows from Infogroup that have SIC Codes that match
# the desired input SIC Code

def sic_matches_df(mst_df, sic_code):
    df_sic = pd.DataFrame(columns=df.columns)
    sic_code = str(sic_code)
    
    for i in range(len(df)):
        if (  (df.iloc[i]['SIC CODE'].__contains__(sic_code) ) |
              (df.iloc[i]['SIC CODE 1'].__contains__(sic_code)) |
              (df.iloc[i]['SIC CODE 2'].__contains__(sic_code)) |
              (df.iloc[i]['SIC CODE 3'].__contains__(sic_code)) |
              (df.iloc[i]['SIC CODE 4'].__contains__(sic_code)) |
              (df.iloc[i]['PRIMARY SIC CODE'].__contains__(sic_code))
            ):
            df_sic.loc[len(df_sic.index)] = mst_df.iloc[i]
            
    return df_sic

In [7]:
# create new empty dataframe (to append to) containing only SIC Codes that we 0291
# df_sic_0291 = pd.DataFrame(columns=df.columns)
df_sic_0291 = sic_matches_df(df, '0291')


In [8]:
df_sic_0291.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,...,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,CHURCH RANCH,20009 HIGHWAY 72,ARVADA,CO,80007,8236,059,303,2,A,...,5,060500,3,039.869196,-105.230716,P,19740,2,216,08059
1,SAN PABLITO RANCH,7660 WOODWAY DR,HOUSTON,TX,77063,1533,201,713,2,A,...,7,430300,1,029.752743,-095.503489,P,26420,2,288,48201
2,REMINGTON CREEK RANCH SALES,1015 STABLE SIDE CT,HOUSTON,TX,77073,6409,201,281,2,A,...,7,240702,2,029.966813,-095.393341,P,26420,2,288,48201
3,MOUNTAIN SHADOWS RANCH,3135 SHADY HOLLOW LN,JAMUL,CA,91935,2236,073,619,2,A,...,5,021302,2,032.737980,-116.818903,P,41740,2,000,06073
4,SMOKE TREE RANCH,1850 SMOKE TREE LN,PALM SPRINGS,CA,92264,9270,065,760,2,E,...,6,044807,1,033.797426,-116.521896,P,40140,2,348,06065


In [7]:
FILENAME = "../data/code_0291.csv"
df_sic_0291.to_csv(FILENAME)

In [8]:
# create new empty dataframe (to append to) containing only SIC Codes: 0241
df_sic_0241 = sic_matches_df(df, '0241')


In [9]:
df_sic_0241.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,...,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,MOSSING'S DAIRY LLC,47447 233A ST,EGAN,SD,57024,6439,101,605,2,B,...,2,959600,2,043.998130,-096.721610,P,00000,NaN,000,46101
1,LONG DAIRY FARM INC,27164 477TH AVE,HARRISBURG,SD,57032,8210,083,605,2,A,...,5,010107,3,043.450902,-096.667723,P,43620,2,000,46083
2,CYPHER DAIRY CONSULTING LLC,26443 461ST AVE,HARTFORD,SD,57033,6703,099,605,2,A,...,4,010300,2,043.551800,-096.992924,P,43620,2,000,46099
3,UNITED DAIRY INC,6 GARVIN DAIRY RD,FAIRMONT,WV,26554,5058,049,304,2,D,...,7,021000,2,039.481453,-080.082759,P,21900,1,390,54049
4,DAIRYNET INC,2301 RESEARCH PARK WAY # 155,BROOKINGS,SD,57006,1724,011,605,2,B,...,7,958900,5,044.320347,-096.764656,P,15100,1,000,46011


In [10]:
FILENAME = "../data/code_0241.csv"
df_sic_0241.to_csv(FILENAME)

In [6]:
df_sic_5144 = sic_matches_df(df, '5144')

In [7]:
df_sic_5144.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,...,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,SONSTEGARD FOODS CO,5005 S BUR OAK PL,SIOUX FALLS,SD,57108,2228,099,605,2,B,...,6,010102,4,043.499985,-096.762425,0,43620,2,000,46099
1,STEMCOR,1 E BROWARD BLVD # 1599,FORT LAUDERDALE,FL,33301,2040,011,954,2,A,...,6,042500,3,026.122817,-080.143243,P,33100,2,370,12011
2,ROCK RIDGE FARM,752 STATE RD,RICHMOND,MA,01254,5246,003,413,2,A,...,2,935100,3,042.409203,-073.355258,P,38340,2,000,25003
3,CHICK-A-RAY POULTRY,24877 THOMPSON RD,ALBEMARLE,NC,28001,7467,167,704,2,A,...,7,931202,1,035.319232,-080.255050,P,10620,1,172,37167
4,MAR-JAC POULTRY INC,1301 JAMES ST,HATTIESBURG,MS,39401,4570,035,601,2,H,...,7,010500,1,031.307197,-089.278029,P,25620,2,279,28035


In [8]:
FILENAME = "../data/code_5144.csv"
df_sic_5144.to_csv(FILENAME)

In [9]:
df_sic_5154 = sic_matches_df(df, '5154')

In [10]:
df_sic_5154.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,...,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,MADISON LIVESTOCK SALES CO,1209 E CENTER ST,MADISON,SD,57042,NaN,079,605,2,A,...,5,960200,4,044.000500,-097.118500,X,00000,NaN,000,46079
1,UNITED PRODUCERS INC,3491 E STATE ROAD 44,RUSHVILLE,IN,46173,7846,139,765,2,B,...,6,974400,3,039.615453,-085.379248,P,00000,NaN,000,18139
2,LAFLEUR BROTHERS LIVESTOCK,33022 482ND AVE,JEFFERSON,SD,57038,6800,127,605,2,B,...,3,020300,1,042.602220,-096.565863,P,43580,2,000,46127
3,TYSON FOODS INC,47283 SD HIGHWAY 34,COLMAN,SD,57017,6543,101,605,2,A,...,3,959600,2,043.978463,-096.753100,P,00000,NaN,000,46101
4,MENNO LIVESTOCK AUCTION,602 S PEARL ST,MENNO,SD,57045,2064,067,605,2,D,...,2,968600,3,043.234794,-097.574566,P,00000,NaN,000,46067


In [11]:
FILENAME = "../data/code_5154.csv"
df_sic_5154.to_csv(FILENAME)

In [12]:
df_sic_0191 = sic_matches_df(df, '0191')

In [13]:
df_sic_0191.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,...,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,OLD RIVER FARM,50 MILLER RD,MIDDLEFIELD,CT,06455,1229,007,860,2,A,...,4,580100,4,041.500023,-072.699730,P,25540,2,278,09007
1,FELIX CHAC CHUO FARMS INC,89785 80TH AVE,THERMAL,CA,92274,8906,065,760,2,B,...,6,045605,5,033.468129,-116.098759,P,40140,2,348,06065
2,C & N PETERSON FARMS INC,30310 455TH AVE,WAKONDA,SD,57073,6300,027,605,2,A,...,2,965800,3,042.992997,-097.096458,P,46820,1,000,46027
3,E-I-E-I-O FARM LLC,1003 NEWFIELD ST,MIDDLETOWN,CT,06457,1817,007,860,2,A,...,7,541200,3,041.586472,-072.674690,P,25540,2,278,09007
4,DRAGO INDIANA,6147 N COUNTY ROAD 500 W,MULBERRY,IN,46058,9445,023,765,2,D,...,3,950300,3,040.377054,-086.596185,P,23140,1,320,18023


In [14]:
FILENAME = "../data/code_0191.csv"
df_sic_0191.to_csv(FILENAME)

In [32]:
USEFUL_COLS = [
    "COMPANY",
    "ADDRESS LINE 1",
    "CITY",
    "STATE",
    ]
SIC_COLS = [
    "PRIMARY SIC CODE", 
    'SIC CODE 1', 
    'SIC CODE 2', 
    'SIC CODE 3',
    'SIC CODE 4'
    ]
NAICS_COLS = [
    "NAICS CODE",
    "PRIMARY NAICS CODE"
    ]
DESCRIPTION_COLS = [
    # "NAICS8 DESCRIPTIONS", # only in 2021
    # "SIC6_DESCRIPTIONS (PRIMARYSIC)" # only in 2021
    "SIC6_DESCRIPTIONS (SIC)"
]

In [33]:
poultry_2022 = pd.read_csv("../data/poultry_plants_2022.csv")
poultry_2022.iloc[5]['SIC CODE']

poultry_2022[USEFUL_COLS + SIC_COLS + DESCRIPTION_COLS]

# poultry_2022.columns


,COMPANY,ADDRESS LINE 1,CITY,STATE,PRIMARY SIC CODE,SIC CODE 1,SIC CODE 2,SIC CODE 3,SIC CODE 4,SIC6_DESCRIPTIONS (SIC)
0,COOTERS FARM,1784 TUCK WILKES RD,NOXAPATER,MS,201501,NaN,NaN,NaN,NaN,POULTRY FARMS
1,TYSON FOODS INC,47283 SD HIGHWAY 34,COLMAN,SD,201104,25401.0,201501.0,204803.0,207702.0,LIVESTOCK-DEALERS (WHLS)
2,SIMMONS FOODS INC,2101 TWIN CIRCLE DR,VAN BUREN,AR,201501,NaN,NaN,NaN,NaN,NaN
3,TYSON FOODS INC,605 235-3301 #DD813,NORTH SIOUX CITY,SD,25401,201501.0,204803.0,207702.0,209903.0,MEAT PRODUCTS (MFRS)
4,MAR-JAC POULTRY INC,1301 JAMES ST,HATTIESBURG,MS,201501,NaN,NaN,NaN,NaN,POULTRY-WHOLESALE
...,...,...,...,...,...,...,...,...,...,...
715,PILGRIMS PRIDE,2237 WHITLEY RD E,WILSON,NC,201501,NaN,NaN,NaN,NaN,TRUCKING-CONTRACT HAULING
716,2-C'S ENTERPRISES,961 WHITES BOTTOM RD,BURKESVILLE,KY,201501,NaN,NaN,NaN,NaN,NaN
717,DAYBREAK FOODS INC,533 E TYRANENA PARK RD,LAKE MILLS,WI,541105,201501.0,NaN,NaN,NaN,FEDERAL GOVERNMENT CONTRACTORS
718,BELLARD'S POULTRY,405 S BULLARD ST,OPELOUSAS,LA,201501,549911.0,549907.0,571912.0,NaN,POULTRY FARMS


## New Dataframe containing only certain states

In [5]:
certain_states = ['IA', 'OK', 'MO', 'OK', 'AL', 'LA', 'MS', 'IL', 'IN', 'OH', 'KY', 'TN', 'AR', 'NC', 'SC', 'GA']

In [4]:
df = dd.read_csv(INFOGROUP_2022[0], dtype=str, encoding='unicode_escape')
df.columns = df.columns.str.upper() # force all columns to be uppercase
df.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,...,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,MIRAGE DRAPERIES,4731 E SUNNY DUNES RD,PALM SPRINGS,CA,92264,1536,065,760,2,A,...,6,044804,1,033.812119,-116.494500,0,40140,2,348,06065
1,NaN,11700 OLIO RD,FISHERS,IN,46037,7618,057,317,1,A,...,7,110808,1,039.959999,-085.921268,P,26900,2,294,18057
2,STONE RESIN SURFACING LLC,1 MAYFLOWER PL,MILFORD,CT,06460,4520,009,203,2,D,...,7,150200,2,041.207739,-073.070136,P,35300,2,408,09009
3,NaN,2121 7TH ST,PARKERSBURG,WV,26101,3803,107,740,1,NaN,...,7,000702,1,039.269405,-081.520599,P,37620,2,425,54107
4,IRONWOOD STATE PRISON,19005 WILEYS WELL RD,BLYTHE,CA,92225,2287,065,760,2,A,...,7,046900,1,033.570065,-114.898031,0,40140,2,348,06065


In [7]:
df_certain_states = df[df['STATE'].isin(certain_states)]

In [8]:
df_certain_states = df_certain_states.compute()

In [9]:
len(df_certain_states)

4651574

In [10]:
df_certain_states['STATE'].value_counts()

STATE
IL    606453
OH    561009
NC    526234
GA    479516
IN    306130
TN    305878
MO    294764
LA    260162
SC    229292
KY    220252
AL    216301
OK    208352
IA    170058
AR    138974
MS    128199
Name: count, dtype: int64

In [11]:
df_certain_states.to_csv("../data/certain_states.csv") # save as CSV file